## 3. Flask Annotator App for Concordance Line Annotation & Analysis

- This notebook demonstrates how to use an AI prompt engineering Flask App for quick annotation of concordance lines obtained from KWIC searches on various corpora (e.g., COCA, GloWbE, EcoLexicon).

In [ ]:
ls -la ../../annotator_app/

In [ ]:
import sys
import subprocess
import threading
from pathlib import Path
import time

# Run the Flask app at ./annotator_app/app.py without blocking the notebook kernel.
# This cell starts the script as a subprocess and streams its stdout/stderr.

app_path = Path("../..//annotator_app/app.py").resolve()
if not app_path.exists():
    raise FileNotFoundError(f"{app_path} not found. Check the path and try again.")

# Start the app using the same Python interpreter as the notebook
proc = subprocess.Popen(
    [sys.executable, str(app_path)],
    cwd=str(app_path.parent),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1
)

print(f"Started process PID={proc.pid} for {app_path}")

# Background thread to stream process output to the notebook
def _stream_output(p):
    try:
        for line in p.stdout:
            print(line, end="")
    except Exception as e:
        print("Error streaming output:", e)

thread = threading.Thread(target=_stream_output, args=(proc,), daemon=True)
thread.start()

# Convenience: helper functions to check status and stop the process
def is_running():
    return proc.poll() is None

def stop_app(timeout=5):
    if not is_running():
        print("Process is not running.")
        return
    print("Terminating process...")
    proc.terminate()
    try:
        proc.wait(timeout=timeout)
    except subprocess.TimeoutExpired:
        print("Process did not exit, killing...")
        proc.kill()
    print("Process stopped.")

# Example usage:
# is_running()          -> returns True if app is still running
# stop_app()            -> gracefully stop the app (force-kills if needed)
# proc.pid              -> access the process id
#
# Note: If the Flask app binds to a port (e.g., 5000), open http://localhost:5000 in your browser.
time.sleep(0.1)  # small sleep so the cell returns after process started

_______________________
## Troubleshoot

- Restart the kernel if the Flask app does not start properly.
- Check for running PIDs on the same port:

In [ ]:
# check what is running on port 5001
!lsof -i :5001

In [ ]:
!kill -9 $(lsof -t -i:5000) || echo "No process on port 5000"

In [ ]:
!kill -0 {proc.pid} && echo "Process {proc.pid} is running." || echo "Process {proc.pid} is not running."